In [2]:
%pip install -e /Users/yousuf/Documents/ucl-msc/meta-misinformation-detection

distutils: /usr/local/lib/python3.9/site-packages
sysconfig: /usr/local/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages
distutils: /usr/local/lib/python3.9/site-packages
sysconfig: /usr/local/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages
distutils: /usr/local/include/python3.9/UNKNOWN
sysconfig: /usr/local/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/include/python3.9
distutils: /usr/local/bin
sysconfig: /usr/local/opt/python@3.9/Frameworks/Python.framework/Versions/3.9/bin
distutils: /usr/local
sysconfig: /usr/local/opt/python@3.9/Frameworks/Python.framework/Versions/3.9
user = False
home = None
root = None
prefix = None
Obtaining file:///Users/yousuf/Documents/ucl-msc/meta-misinformation-detection
  Attempting uninstall: metaphor
    Found existing installation: metaphor 0.0.1
    Uninstalling metaphor-0.0.1:
      Successfully uninstalled metaphor-0.0.1
  distutils: /usr/local/lib/python3.9/site-pac

In [1]:
from metaphor.models.common import Tokenizer, WordEmbedding, Glove, MeanPooler
sentences = ["Hello", "Hello World"]
t = Tokenizer()
t_sentences = t(sentences)
embedder = WordEmbedding(Glove(t).weights, MeanPooler(t))
pooled_embedding = embedder(t_sentences)

/usr/local/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


KeyboardInterrupt: 

In [1]:
import numpy as np 
np.random.seed(0)
np.random.randint(4)

0

In [5]:
np.random.seed(0)
x = np.arange(10)
np.random.permutation(x)

array([2, 8, 4, 9, 1, 6, 7, 3, 0, 5])

In [5]:
import re
text = "#Germanwings co-pilot suffered serious depression 6 years ago: German newspaper http://t.co/LXDzBdNdMB http://t.co/yTUNdHyEJY"
regex = r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"
text = re.sub(regex, "", text)
text.strip()

'#Germanwings co-pilot suffered serious depression 6 years ago: German newspaper'

In [3]:
import pandas as pd
df = pd.read_csv("/Users/yousuf/Documents/ucl-msc/meta-misinformation-detection/data/pheme/processed-pheme.csv")

In [7]:
df = df.dropna()
topics = sorted(list(set(df["topic"])))
len(topics)

9

In [12]:
topics = ['a', 'b', 'c']
data = {'topic': ['b','c','a','a','b']}
topic_indxs = [[i for i in range(len(data['topic'])) if topics.index(data['topic'][i]) == j] for j in range(len(topics))]

In [13]:
topic_indxs

[[2, 3], [0, 4], [1]]

In [16]:
topic_indxs = [
        [
            i
            for i in range(len(data["topic"]))
            if topics.index(data["topic"][i]) == j
        ]
        for j in range(len(topics))
    ]

In [17]:
topic_indxs

[[2, 3], [0, 4], [1]]

In [29]:
def filter_indices(t):
    return [[y for y in x if y in t] for x in topic_indxs]

ti = [0,1,2]
indices = [
    filter_indices(t)
    for t in [ti]
]
z = [
    [
        t for t in i
    ]
    for i in indices
]

In [32]:
z[0]

[[2], [0], [1]]

In [37]:
import numpy as np
x = np.array([1,2,3])
x[np.array([])]

IndexError: arrays used as indices must be of integer (or boolean) type

In [67]:
global_explainers = {0:{'apple': [0.1,0.6,0.8,0.9], 'beta':[0.1,0.4], 'dog':[0.5]}, 1:{'apple':[-0.5,0.2],'gamma':[0.3,0.1]},2:{'dog':[0.1], 'apple':[-5]}}
lime_scores_per_class = (
    {}
)  # a map from word to [false lime score, unverified lime score, true lime score]
for label in range(3):
    for word in global_explainers[label].keys():
        scores = np.array(global_explainers[label][word])
        lime_score = np.sqrt(np.maximum(0,np.sum(scores)))
#         lime_score = np.sqrt(np.sum(np.abs(scores)))
        if not (word in lime_scores_per_class):
            lime_scores_per_class[word] = np.zeros((3))
        lime_scores_per_class[word][label] = lime_score
        
print(lime_scores_per_class)
N = len(lime_scores_per_class.keys())
p_cj = np.zeros((N, 3))
lime_scores = np.zeros((N, 3))
for i, word in enumerate(lime_scores_per_class.keys()):
    p_cj[i] = lime_scores_per_class[word] / lime_scores_per_class[word].sum()
    print(lime_scores_per_class[word])
    lime_scores[i,:] = lime_scores_per_class[word]

# compute entropy
print(p_cj)
H = (-p_cj * np.log(p_cj + (10 ** -8))).sum(axis=1)
coeff = (1 - (H - np.min(H)) / (np.max(H) - np.min(H))) 
coeff = coeff.reshape((coeff.shape[0],1))
lime_scores = coeff * lime_scores

for i, word in enumerate(lime_scores_per_class.keys()):
    lime_scores_per_class[word] = lime_scores[i]

{'apple': array([1.54919334, 0.        , 0.        ]), 'beta': array([0.70710678, 0.        , 0.        ]), 'dog': array([0.70710678, 0.        , 0.31622777]), 'gamma': array([0.        , 0.63245553, 0.        ])}
[1.54919334 0.         0.        ]
[0.70710678 0.         0.        ]
[0.70710678 0.         0.31622777]
[0.         0.63245553 0.        ]
[[1.         0.         0.        ]
 [1.         0.         0.        ]
 [0.69098301 0.         0.30901699]
 [0.         1.         0.        ]]


In [68]:
(1 - (H - np.min(H)) / (np.max(H) - np.min(H)))

array([1., 1., 0., 1.])

In [69]:
H

array([-9.99999989e-09, -9.99999989e-09,  6.18311863e-01, -9.99999989e-09])

In [64]:
lime_scores_per_class

{'apple': array([0., 0., 0.]),
 'beta': array([0.70710678, 0.        , 0.        ]),
 'dog': array([0.03225417, 0.        , 0.0144245 ]),
 'gamma': array([0.        , 0.63245553, 0.        ])}

In [76]:
import numpy as np
import scipy.stats
x = np.array([1,2,3,3])
scipy.stats.mode(x)

ModuleNotFoundError: No module named 'unittest'

In [77]:
import torch

NameError: name '_C' is not defined

In [79]:
from metaphor.data import Pheme
Pheme

ImportError: cannot import name 'Pheme' from 'metaphor.data' (unknown location)

In [2]:
from metaphor.data.loading.data_loader import (
    MisinformationPheme,
    TopicPheme,
    PerTopicMisinformation,
    HardPheme
)

In [6]:
import torch
pheme_path = "/Users/yousuf/Documents/ucl-msc/meta-misinformation-detection/data/pheme/processed-pheme.csv"
pheme = MisinformationPheme(
    file_path=pheme_path,
    tokenizer=lambda x: x,
    embedder=lambda x: torch.zeros((len(x), 200)),
)

In [11]:
len(pheme.data)

2402

In [29]:
gr = 100*pheme.data.groupby('topic').count()['veracity']/len(pheme.data)

In [30]:
from itertools import chain, combinations

def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))


In [46]:
gr = 100*pheme.data.groupby('topic').count()['veracity']/len(pheme.data)
pset = list(powerset(gr.keys()))
topics = []
for combo in pset:
    sum = 0
    for x in combo:
        sum += gr[x]
    if sum > 15 and sum < 25:
        print(f'{combo} {sum}')
        topics.append(combo)

('charliehebdo',) 19.06744379683597
('ottawashooting',) 19.567027477102414
('sydneysiege',) 21.73189009159034
('charliehebdo', 'ebola-essien') 19.65029142381349
('charliehebdo', 'gurlitt') 21.60699417152373
('charliehebdo', 'putinmissing') 24.31307243963364
('ebola-essien', 'ottawashooting') 20.149875104079932
('ebola-essien', 'sydneysiege') 22.31473771856786
('ferguson', 'germanwings-crash') 21.731890091590344
('ferguson', 'prince-toronto') 21.35720233139051
('ferguson', 'putinmissing') 17.069109075770193
('germanwings-crash', 'prince-toronto') 19.442131557035804
('germanwings-crash', 'putinmissing') 15.154038301415486
('gurlitt', 'ottawashooting') 22.106577851790174
('gurlitt', 'sydneysiege') 24.2714404662781
('ottawashooting', 'putinmissing') 24.812656119900083
('charliehebdo', 'ebola-essien', 'gurlitt') 22.18984179850125
('charliehebdo', 'ebola-essien', 'putinmissing') 24.895920066611158
('ebola-essien', 'ferguson', 'germanwings-crash') 22.314737718567862
('ebola-essien', 'ferguson

In [52]:
np.random.choice(topics)

<ipython-input-52-0a9f2988f4a4>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.random.choice(topics)


('charliehebdo', 'ebola-essien', 'gurlitt')

In [26]:
pset[1]
gr['charliehebdo']

458

In [21]:
gr.keys()

Index(['charliehebdo', 'ebola-essien', 'ferguson', 'germanwings-crash',
       'gurlitt', 'ottawashooting', 'prince-toronto', 'putinmissing',
       'sydneysiege'],
      dtype='object', name='topic')

In [45]:
import numpy as np
indxs = [x in ['ferguson', 'germanwings-crash'] for x in pheme.data['topic']]
np.arange(len(pheme.data))[indxs]

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [59]:
x = torch.tensor([1,2,3])
y = torch.tensor([1,5,3])
x.eq(y).float().mean().item()

0.6666666865348816

In [62]:
x = np.array([1,2,3,4,5,6])
x[-2:]=1
x

array([1, 2, 3, 4, 1, 1])

In [68]:
target = 0
x = torch.tensor([[1,0,0], [0,1,1]])
x *= -1
x[:,target] *= -1
torch.sum(x,dim=1)



tensor([ 1, -2])

In [65]:
x[:,[1,2]]

tensor([[0, 0],
        [1, 1]])